In [29]:
# ================================================================
# Train Scheduling (two-mode: non-failed R_ok vs. failed R_aff)
# - Non-failed: path fixed, delay via wait-arcs only
#   (NO cancel/re-route/truncate variables/constraints created)
#   + Horizon-safe flow with horizon sink at t=T  # <<< FIX
# - Failed: cancel/re-route/truncate allowed with neighborhood (K-hop, Θ)
# - Global headway: same (i->j) cannot depart at same time
# ================================================================

import json, os, heapq
from collections import defaultdict, deque
import gurobipy as gp
import pandas as pd

# ---------------------- USER PARAMS -----------------------------
BASE_DIR   = r"D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full\json"
edge_fp    = os.path.join(BASE_DIR, "edges.json")
route_fp   = os.path.join(BASE_DIR, "routes_nodes.json")
demand_fp  = os.path.join(BASE_DIR, "demand_full.json")
dept_fp    = os.path.join(BASE_DIR, "dep_time.json")

T          = 24          # time horizon
HEADWAY_ON = True        # global headway on same (i->j, t)
K_HOP      = 3           # neighborhood K
THETA      = 8           # neighborhood Θ
BIG_M      = 10**6
W_ROUTE    = 100
w1, w2, w3 = 1000, 1000, 10

failed_edges = {"e43r"}     # e.g. {"e43r"} / set()
SINK = "SINK"
# ---------------------- LOAD INPUTS -----------------------------
with open(edge_fp, encoding="utf-8") as f:
    edges_raw = json.load(f)  # eid -> (src, dst, tau)
edges = {eid: (src, dst, int(tau)) for eid, (src, dst, tau) in edges_raw.items()}

with open(route_fp, encoding="utf-8") as f:
    routes_nodes = json.load(f)   # tr -> [n0, n1, ... nL]

with open(dept_fp, encoding="utf-8") as f:
    dep_time = {tr: int(t) for tr, t in json.load(f).items()}

with open(demand_fp, encoding="utf-8") as f:
    dem_raw = json.load(f)        # tr -> [(o,d,q), ...]
demand = {tr: [(o, d, float(q)) for o, d, q in lst] for tr, lst in dem_raw.items()}

trains = list(routes_nodes)
nodes  = {n for _, (s, d, _) in edges.items() for n in (s, d)}
nodes.add(SINK)

In [30]:

# ---------------------- BASIC MAPS ------------------------------
uv2eids = defaultdict(list)
tau_of  = {}
adj_out = defaultdict(list)
for eid, (u, v, tau) in edges.items():
    uv2eids[(u, v)].append(eid)
    tau_of[eid] = tau
    adj_out[u].append((v, eid, tau))

def planned_eids_of(tr):
    eids = []
    path = routes_nodes[tr]
    for u, v in zip(path[:-1], path[1:]):
        cand = uv2eids.get((u, v))
        if not cand:
            raise KeyError(f"[{tr}] missing edge ({u}->{v}) in edges.json")
        eids.append(cand[0])
    return eids

planned_eids = {tr: planned_eids_of(tr) for tr in trains}

# ---------------------- SCHEDULE (planned arrivals) -------------
sched = {}
for tr, path in routes_nodes.items():
    t = dep_time[tr]
    arr = {path[0]: t}
    for u, v in zip(path[:-1], path[1:]):
        eid = uv2eids[(u, v)][0]
        t += tau_of[eid]
        arr[v] = t
    sched[tr] = arr

# rem_sched[tr][n] = n에서 r_d까지의 계획 잔여시간
rem_sched = {
    tr: { n: (sched[tr][routes_nodes[tr][-1]] - sched[tr][n]) for n in routes_nodes[tr] }
    for tr in trains
}
# ---------------------- AFFECTED FLAG / ALLOW STOP --------------
blocked = {
    tr: int(any(e in failed_edges for e in planned_eids[tr]))
    for tr in trains
}
R_ok  = [tr for tr in trains if blocked[tr] == 0]
R_aff = [tr for tr in trains if blocked[tr] == 1]

allow_stop = {}
reach = {}
for tr in trains:
    path = routes_nodes[tr]
    eids = planned_eids[tr]
    L = len(path) - 1
    t_sched_term = sched[tr].get(path[-1], BIG_M)
    reach[tr] = int(t_sched_term <= T)
    suffix_failed = [0]*(L+1)
    hit = 0
    for i in range(L-1, -1, -1):
        if eids[i] in failed_edges:
            hit = 1
        suffix_failed[i] = hit
    for i in range(1, L+1):
        n = path[i]
        if i == L:
            allow = 1
        elif reach[tr] == 0:
            allow = 1
        else:
            allow = suffix_failed[i]
        allow_stop[(tr, n)] = int(allow)


In [31]:
# ---------------------- NEIGHBORHOOD (K-hop & Θ) ----------------
def k_hop_nodes_from_path(path_nodes, K):
    if K <= 0:
        return set(path_nodes)
    seen = set(path_nodes)
    frontier = list(path_nodes)
    for _ in range(K):
        nxt = []
        for u in frontier:
            for v, eid, tau in adj_out[u]:
                if v not in seen:
                    seen.add(v); nxt.append(v)
        frontier = nxt
        if not frontier:
            break
    return seen

def dijkstra_cutoff(src, cutoff):
    dist = {}
    pq = [(0, src)]
    while pq:
        d, u = heapq.heappop(pq)
        if d > cutoff: break
        if u in dist: continue
        dist[u] = d
        for v, eid, tau in adj_out[u]:
            nd = d + tau
            if nd <= cutoff and v not in dist:
                heapq.heappush(pq, (nd, v))
    return dist

def anchored_timeset(tr, anchors, t_dep, t_sched_term, THETA):
    best_time = {}
    for a in anchors:
        t_to_anchor = sched[tr][a] - t_dep
        cutoff = (t_sched_term + THETA) - (t_dep + t_to_anchor)
        if cutoff < 0:
            continue
        dist = dijkstra_cutoff(a, cutoff)
        for v, dv in dist.items():
            anchored = t_dep + t_to_anchor + dv
            if anchored <= t_sched_term + THETA:
                if v not in best_time or anchored < best_time[v]:
                    best_time[v] = anchored
    return best_time

def neighborhood_nodes_E(tr, K, THETA):
    path = routes_nodes[tr]
    anchors = list(path)
    t_dep   = dep_time[tr]
    t_term  = sched[tr].get(path[-1], BIG_M)

    V_k = k_hop_nodes_from_path(anchors, K)
    best_time = anchored_timeset(tr, anchors, t_dep, t_term, THETA)
    V_t = {v for v, _ in best_time.items()}

    V_nbh = (V_k & V_t) | set(anchors)         # anchors always in  # <<< FIX: anchors 포함 보장
    E_nbh = []
    for eid, (u, v, tau) in edges.items():
        if eid in failed_edges: 
            continue
        if (u in V_nbh) and (v in V_nbh):
            E_nbh.append(eid)
    return V_nbh, E_nbh

In [35]:
# ---------------------- ARC LISTS PER TRAIN ---------------------
arc_list_r   = {}
arc_idx_r    = {}
in_arcs_r    = {}
out_arcs_r   = {}
node_in_real = {}
node_out_real= {}


def build_arc_structures():
    for tr in trains:
        arcs = []
        inA  = defaultdict(list)
        outA = defaultdict(list)
        inR  = defaultdict(list)
        outR = defaultdict(list)

        path = routes_nodes[tr]
        r_o, r_d = path[0], path[-1]

        # --- R_ok arc generation (build_arc_structures 내부) ---
        if blocked[tr] == 0:
            planned_set = set(planned_eids[tr])
            path_nodes  = routes_nodes[tr]
            r_d = path_nodes[-1]

            # (A) real arcs: (u^t -> v^{t+tau})는 "t + rem_sched[u] <= T"일 때만 생성
            for eid in planned_set:
                u, v, tau = edges[eid]
                for t in range(0, T - tau + 1):
                    if t + rem_sched[tr][u] <= T:
                        arcs.append((f"{u}^{t}", f"{v}^{t+tau}", eid, tau, t, t+tau, "real"))
            # (B) wait arcs: 경로 노드 전 시간에 허용
            for n in path_nodes:
                for t in range(0, T):
                    arcs.append((f"{n}^{t}", f"{n}^{t+1}", f"w_{n}", 1, t, t+1, "wait"))
            # (C) 종착 더미: r_d에서 t=0..T
            for t in range(0, T+1):
                arcs.append((f"{r_d}^{t}", f"{SINK}^{t}", f"dummy_{r_d}", 0, t, t, "dummy"))
            # (D) "늦어버린 상태" 더미: t + rem[n] > T 인 상태에만 (n^t -> SINK^t)
            for n in path_nodes:
                cutoff = T - rem_sched[tr][n]
                for t in range(cutoff+1, T+1):  # cutoff+1..T 가 "늦음"
                    arcs.append((f"{n}^{t}", f"{SINK}^{t}", f"late_{n}", 0, t, t, "dummy"))

        else:
            # -------- R_aff: neighborhood subgraph
            V_nbh, E_nbh = neighborhood_nodes_E(tr, K_HOP, THETA)
            # real arcs in neighborhood
            for eid in E_nbh:
                if eid in failed_edges:
                    continue
                u, v, tau = edges[eid]
                for t in range(0, T - tau + 1):
                    fr, to = f"{u}^{t}", f"{v}^{t+tau}"
                    arcs.append((fr, to, eid, tau, t, t+tau, "real"))
            # wait arcs on V_nbh
            for n in V_nbh:
                for t in range(0, T):
                    fr, to = f"{n}^{t}", f"{n}^{t+1}"
                    arcs.append((fr, to, f"w_{n}", 1, t, t+1, "wait"))
            # allowed dummy stops
            for n in path[1:]:
                if allow_stop.get((tr, n), 0) == 1:
                    for t in range(0, T+1):
                        fr, to = f"{n}^{t}", f"{SINK}^{t}"
                        arcs.append((fr, to, f"dummy_{n}", 0, t, t, "dummy"))

        # index & IO maps
        idx_map = {a: i for i, a in enumerate(arcs)}
        for k, (fr, to, eid, tau, t0, t1, kind) in enumerate(arcs):
            nf, tf = fr.split("^"); tf = int(tf)
            nt, tt = to.split("^"); tt = int(tt)
            outA[(nf, tf)].append(k)
            inA [(nt, tt)].append(k)
            if kind == "real":
                outR[nf].append(k)
                inR [nt].append(k)

        arc_list_r[tr]   = arcs
        arc_idx_r [tr]   = idx_map
        in_arcs_r [tr]   = inA
        out_arcs_r[tr]   = outA
        node_in_real[tr] = inR
        node_out_real[tr]= outR

build_arc_structures()








# 출력코드
# 1) R_ok 가운데 reach==0인 트레인 출력 (이미 힌트에서 본 목록보다 자세히)
bad_Rok = [tr for tr in trains if blocked[tr]==0 and reach[tr]==0]
print("R_ok but sched_term > T (reach==0):", bad_Rok)

for tr in bad_Rok:
    r_o = routes_nodes[tr][0]; r_d = routes_nodes[tr][-1]
    print("\n---", tr)
    print(" dep_time:", dep_time[tr])
    print(" sched_term:", sched[tr].get(r_d, None))
    print(" rem_sched at origin:", rem_sched[tr][r_o])
    # outgoing arcs at origin time
    out_idx = out_arcs_r[tr].get((r_o, dep_time[tr]), [])
    print(" out arcs at origin (indices):", out_idx)
    print(" out arc kinds at origin:", [arc_list_r[tr][k][6] for k in out_idx])
    # dummy indices
    idx_rd   = [k for k,a in enumerate(arc_list_r[tr]) if a[2] == f"dummy_{r_d}"]
    idx_late = [k for k,a in enumerate(arc_list_r[tr]) if a[6]=="dummy" and str(a[2]).startswith("late_")]
    print(" idx_rd (dummy_r_d):", idx_rd)
    print(" idx_late (late dummy):", idx_late)
    # Are there any real arcs at all?
    real_cnt = sum(1 for a in arc_list_r[tr] if a[6]=="real")
    print(" real arc count for this tr:", real_cnt)

# 출력용임
import json
def extract_neighborhoods(tr_list):
    res = {}
    for tr in tr_list:
        try:
            V_nbh, E_nbh = neighborhood_nodes_E(tr, K_HOP, THETA)
            anchors = list(routes_nodes[tr])
            t_dep = dep_time[tr]
            t_term = sched[tr].get(routes_nodes[tr][-1], None)
            best_time = anchored_timeset(tr, anchors, t_dep, t_term, THETA)
        except Exception as e:
            res[tr] = {"error": str(e)}
            continue

        planned = set(planned_eids.get(tr, []))
        inter = planned & set(E_nbh)
        bt_sample = sorted(best_time.items(), key=lambda x: x[1])[:10]

        res[tr] = {
            "V_nbh": sorted(list(V_nbh)),
            "E_nbh": sorted(list(E_nbh)),
            "origin_in": routes_nodes[tr][0] in V_nbh,
            "dest_in": routes_nodes[tr][-1] in V_nbh,
            "planned_count": len(planned),
            "planned_in_nbh": len(inter),
            "planned_missing_sample": list(planned - set(E_nbh))[:10],
            "best_time_sample": bt_sample
        }
    return res

# R_aff 대상만 추출
aff_trains = [tr for tr in trains if blocked[tr] == 1]

# neighborhood 정보 추출
nbh_map_aff = extract_neighborhoods(aff_trains)

# JSON으로 예쁘게 출력
import json
print(json.dumps(nbh_map_aff, ensure_ascii=False, indent=2))

# 확인용: 각 tr의 blocked 상태도 같이 찍고 싶다면
for tr in aff_trains:
    print(f"{tr} | blocked={blocked[tr]} (R_aff)")


R_ok but sched_term > T (reach==0): ['경부고속철도2_10', '경부고속철도2_11', '경부선1_3', '경부선1_4', '경부선1_5', '경부선1_6', '경부선2_3', '경부선2_4', '경부선2_5', '경부선2_6', '중앙선1_3', '중앙선1_4', '중앙선1_5', '중앙선1_6', '중앙선2_3', '중앙선2_4', '중앙선2_5', '중앙선2_6']

--- 경부고속철도2_10
 dep_time: 9
 sched_term: 25
 rem_sched at origin: 16
 out arcs at origin (indices): [197, 501]
 out arc kinds at origin: ['wait', 'dummy']
 idx_rd (dummy_r_d): [476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 492, 493, 494, 495, 496, 497, 498, 499, 500]
 idx_late (late dummy): [501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556, 557, 558, 559, 560, 561, 562, 563, 564, 565, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577, 578, 579, 580, 581, 582, 583, 584, 585, 586, 587]
 real arc co

In [33]:
# ---------------------- MODEL BUILD -----------------------------
m = gp.Model()
m.Params.OutputFlag = 1

# x variables
x = {}
for tr in trains:
    for k, (fr,to,eid,tau,t0,t1,kind) in enumerate(arc_list_r[tr]):
        vtype = gp.GRB.CONTINUOUS if kind == "wait" else gp.GRB.BINARY
        x[(tr, k)] = m.addVar(lb=0, ub=1, vtype=vtype, name=f"x[{tr},{k}]")

# Arrival times only on OD destinations
t_arr = {}
for tr in trains:
    dest_nodes = {d for (_, d, _) in demand[tr]}
    for n in dest_nodes:
        t_arr[(tr, n)] = m.addVar(lb=0, ub=T+THETA, vtype=gp.GRB.CONTINUOUS, name=f"T[{tr},{n}]")
# (또는 ub=BIG_M도 OK)


# Delay
delta = {(tr, o, d): m.addVar(lb=0, name=f"delta[{tr},{o},{d}]")
         for tr in trains for (o, d, q) in demand[tr]}

# Affected-only vars
h, u_eid, y, s, z = {}, {}, {}, {}, {}
for tr in R_aff:
    h[tr] = m.addVar(vtype=gp.GRB.BINARY, name=f"h[{tr}]")
    P = set(planned_eids[tr])
    real_eids_for_tr = {eid for (_,_,eid,_,_,_,kind) in arc_list_r[tr] if kind=="real"}
    for eid in real_eids_for_tr:
        if eid not in P:
            u_eid[(tr, eid)] = m.addVar(vtype=gp.GRB.BINARY, name=f"u[{tr},{eid}]")
    stop_nodes = { a[0].split("^")[0] for a in arc_list_r[tr] if a[2].startswith("dummy_") }
    for n in stop_nodes:
        y[(tr, n)] = m.addVar(vtype=gp.GRB.BINARY, name=f"y[{tr},{n}]")
    epts = {o for (o,_,_) in demand[tr]} | {d for (_,d,_) in demand[tr]} | {routes_nodes[tr][0]}
    for n in epts:
        s[(tr, n)] = m.addVar(vtype=gp.GRB.BINARY, name=f"s[{tr},{n}]")
    for (o, d, q) in demand[tr]:
        z[(tr, o, d)] = m.addVar(vtype=gp.GRB.BINARY, name=f"z[{tr},{o},{d}]")

m.update()

# ---------------------- CONSTRAINTS -----------------------------

# Departure coupling & flow conservation
for tr in trains:
    outA, inA = out_arcs_r[tr], in_arcs_r[tr]
    r_o = routes_nodes[tr][0]
    t_dep = dep_time[tr]

    idx_out = outA.get((r_o, t_dep), [])
    if len(idx_out) == 0:
        raise RuntimeError(f"[{tr}] No outgoing arc at (origin, dep). Check T or arc generation.")  # <<< FIX guard

    if tr in R_ok:
        m.addConstr(gp.quicksum(x[(tr, k)] for k in idx_out) == 1, name=f"depart_ok[{tr}]")
    else:
        m.addConstr(gp.quicksum(x[(tr, k)] for k in idx_out) == h[tr], name=f"depart_aff[{tr}]")

    keys = set(inA.keys()) | set(outA.keys())
    for (n, t) in keys:
        if n == SINK:
            continue
        inflow  = gp.quicksum(x[(tr, k)] for k in inA.get((n, t), []))
        outflow = gp.quicksum(x[(tr, k)] for k in outA.get((n, t), []))
        if (n == r_o) and (t == t_dep):
            if tr in R_ok:
                m.addConstr(outflow - inflow == 1, name=f"flow_src_ok[{tr},{t}]")
            else:
                m.addConstr(outflow - inflow == h[tr], name=f"flow_src_aff[{tr},{t}]")
        else:
            m.addConstr(inflow == outflow, name=f"flow[{tr},{n},{t}]")

# --- Termination rules (REPLACE this whole R_ok block) ---
for tr in R_ok:
    r_d = routes_nodes[tr][-1]
    idx_rd   = [k for k,a in enumerate(arc_list_r[tr]) if a[2] == f"dummy_{r_d}"]
    idx_late = [k for k,a in enumerate(arc_list_r[tr]) if a[6]=="dummy" and str(a[2]).startswith("late_")]
    m.addConstr(gp.quicksum(x[(tr,k)] for k in (idx_rd + idx_late)) == 1,
                name=f"end_ok_rd_or_late[{tr}]")
    # (있다면) horizon_* 더미는 금지
    for k,a in enumerate(arc_list_r[tr]):
        if a[6]=="dummy" and str(a[2]).startswith("horizon_"):
            x[(tr,k)].UB = 0.0


# 도착시간 지연 연동 추가
for tr in R_ok:
    r_d = routes_nodes[tr][-1]
    for k, a in enumerate(arc_list_r[tr]):
        fr, to, eid, tau, t0, t1, kind = a
        if kind == "dummy" and str(eid).startswith("late_"):
            n = fr.split("^")[0]  # late 더미는 (n^t -> SINK^t)
            m.addConstr(t_arr[(tr, r_d)] >= T + rem_sched[tr][n] - BIG_M*(1 - x[(tr,k)]),
                        name=f"T_late_lb[{tr},{n},{t0}]")

for tr in R_aff:
    cand_nodes = {n for (tr2, n) in y if tr2 == tr}
    m.addConstr(gp.quicksum(y[(tr, n)] for n in cand_nodes) == h[tr], name=f"sumy[{tr}]")
    for n in cand_nodes:
        idx_dum = [k for k, a in enumerate(arc_list_r[tr]) if a[2] == f"dummy_{n}"]
        m.addConstr(gp.quicksum(x[(tr, k)] for k in idx_dum) == y[(tr, n)], name=f"y_dummy[{tr},{n}]")

# Off-route gating
for tr in R_aff:
    P = set(planned_eids[tr])
    for k, a in enumerate(arc_list_r[tr]):
        _, _, eid, tau, t0, t1, kind = a
        if kind == "real" and eid not in P and (tr, eid) in u_eid:
            m.addConstr(x[(tr, k)] <= u_eid[(tr, eid)], name=f"offroute[{tr},{k}]")

# Visit / OD logic
for tr in R_aff:
    r_o = routes_nodes[tr][0]
    if (tr, r_o) in s:
        m.addConstr(s[(tr, r_o)] == h[tr], name=f"s_origin[{tr}]")
    inR, outR = node_in_real[tr], node_out_real[tr]
    for (tr2, n), svar in list(s.items()):
        if tr2 != tr: 
            continue
        idx = inR.get(n, []) + outR.get(n, [])
        flow = gp.quicksum(x[(tr, k)] for k in idx)
        m.addConstr(flow >= svar, name=f"s_ge[{tr},{n}]")
        m.addConstr(flow <= BIG_M * svar, name=f"s_le[{tr},{n}]")
    for (o, d, q) in demand[tr]:
        m.addConstr(z[(tr, o, d)] <= s[(tr, o)], name=f"z_le_o[{tr},{o},{d}]")
        m.addConstr(z[(tr, o, d)] <= s[(tr, d)], name=f"z_le_d[{tr},{o},{d}]")
        m.addConstr(z[(tr, o, d)] >= s[(tr, o)] + s[(tr, d)] - 1, name=f"z_ge_and[{tr},{o},{d}]")
        m.addConstr(z[(tr, o, d)] <= h[tr], name=f"z_le_h[{tr},{o},{d}]")

# Arrival linking & Delay
for tr in trains:
    dests = {d for (_, d, _) in demand[tr]}
    for k, a in enumerate(arc_list_r[tr]):
        fr, to, eid, tau, t0, t1, kind = a
        if kind != "real":
            continue
        n_to, tt = to.split("^"); tt = int(tt)
        if n_to in dests:
            M_time = T + THETA
            m.addConstr(t_arr[(tr, n_to)] >= tt - M_time * (1 - x[(tr, k)]))
            m.addConstr(t_arr[(tr, n_to)] <= tt + M_time * (1 - x[(tr, k)]))
    for (o, d, q) in demand[tr]:
        sched_t = sched[tr].get(d, T)
        m.addConstr(delta[(tr, o, d)] >= t_arr[(tr, d)] - sched_t, name=f"del_ge[{tr},{o},{d}]")
        if tr in R_ok:
            m.addConstr(delta[(tr, o, d)] <= BIG_M, name=f"del_ub_ok[{tr},{o},{d}]")    # 또는 상한 제약 자체를 제거
        else:
            m.addConstr(delta[(tr, o, d)] <= BIG_M * z[(tr, o, d)], name=f"del_ub_aff[{tr},{o},{d}]")

# Global HEADWAY (same i->j, same depart time)
if HEADWAY_ON:
    head_map = defaultdict(list)
    for tr in trains:
        for k, a in enumerate(arc_list_r[tr]):
            fr, to, eid, tau, t0, t1, kind = a
            if kind != "real": 
                continue
            i, tt0 = fr.split("^"); tt0 = int(tt0)
            j, _   = to.split("^")
            head_map[(i, j, tt0)].append((tr, k))
    for key, arr in head_map.items():
        if arr:
            m.addConstr(gp.quicksum(x[(tr, k)] for (tr, k) in arr) <= 1, name=f"head[{key[0]},{key[1]},{key[2]}]")

# ---------------------- OBJECTIVE -------------------------------
# --- (NEW) 열차별 총수요 dict: 이름 충돌 방지용 ---
q_tot_map = {tr: sum(q for *_, q in demand.get(tr, [])) for tr in trains}

# ---------------------- OBJECTIVE -------------------------------
obj = gp.LinExpr()

# 취소 비용 (영향열차만)
for tr in R_aff:
    obj += w1 * q_tot_map[tr] * (1 - h[tr])

# 미제공 비용 (영향열차만)
for tr in R_aff:
    for (o, d, q) in demand[tr]:
        obj += w2 * q * (1 - z[(tr, o, d)])

# 지연 비용 (전체)
for tr in trains:
    for (o, d, q) in demand[tr]:
        obj += w3 * q * delta[(tr, o, d)]

# 재루팅 벌점
for (tr, eid), uvar in u_eid.items():
    obj += W_ROUTE * uvar

m.setObjective(obj, gp.GRB.MINIMIZE)

# ---------------------- SOLVE ----------------------------------
m.optimize()

# ---------- DIAGNOSTICS if not OPTIMAL  ------------------------  # <<< FIX: 원인 파악 유틸
if m.Status != gp.GRB.OPTIMAL:
    print(f"[status] {m.Status}  (2=OPTIMAL, 3=INFEASIBLE, 4=INF_OR_UNBD, 5=UNBOUNDED, 9=TIME_LIMIT)")
    try:
        m.computeIIS()
        m.write("model.lp")
        m.write("model.ilp")
        m.write("model.iis")
        print("IIS written to model.iis / model.ilp")
    except gp.GurobiError as e:
        print("IIS failed:", e)
    # 추가 힌트: R_ok에서 reach==0인 열차 리스트
    bad = [tr for tr in R_ok if reach[tr]==0]
    if bad:
        print("[hint] R_ok trains with sched_terminal > T:", bad)
    raise RuntimeError("Model did not reach OPTIMAL.")

Set parameter OutputFlag to value 1
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 5 3600XT 6-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 144677 rows, 104531 columns and 477064 nonzeros
Model fingerprint: 0xbb656972
Variable types: 44575 continuous, 59956 integer (59956 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  Objective range  [2e-01, 9e+05]
  Bounds range     [1e+00, 3e+01]
  RHS range        [1e+00, 1e+06]
Presolve removed 30372 rows and 20459 columns
Presolve time: 0.07s

Explored 0 nodes (0 simplex iterations) in 0.12 seconds (0.16 work units)
Thread count was 1 (of 12 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -
[status] 3  (2=OPTIMAL, 3=INFEASIBLE, 4=INF_OR_UNBD, 5=UNBOUNDED, 9=TIME_LIMIT)
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (wi

RuntimeError: Model did not reach OPTIMAL.

In [ ]:
# ====== SAFE SNAPSHOT for plotting ======
import pickle, gzip

def freeze_for_plot(trains, arc_list_r, x, demand, failed_edges):
    """Gurobi 객체 제거: 숫자/문자 타입만 남긴 경량 사본"""
    # 1) x: (tr,k) -> 0/1 로 변환
    x_frozen = {}
    for tr in trains:
        for k in range(len(arc_list_r[tr])):
            try:
                v = x[(tr, k)]
                val = float(v.X) if hasattr(v, "X") else float(v)
            except Exception:
                val = 0.0
            if val > 0.5:  # 1만 저장하면 파일이 작아짐 (원하면 모두 저장)
                x_frozen[(tr, k)] = 1
    # 2) arc_list_r: 순수 튜플들만 복사
    arc_list_r_frozen = {
        tr: [(fr, to, str(eid), int(tau), int(t0), int(t1), str(kind))
             for (fr, to, eid, tau, t0, t1, kind) in arc_list_r[tr]]
        for tr in trains
    }
    # 3) demand/failed_edges: 내장형으로
    demand_frozen = {tr: [(str(o), str(d), float(q)) for (o, d, q) in demand.get(tr, [])]
                     for tr in trains}
    failed_edges_list = list(failed_edges)

    return {
        "x": x_frozen,
        "arc_list_r": arc_list_r_frozen,
        "trains": list(trains),
        "demand": demand_frozen,
        "failed_edges": failed_edges_list,
    }

plot_blob = freeze_for_plot(trains, arc_list_r, x, demand, failed_edges)

# 피클 저장 (권장: gzip으로 압축)
with gzip.open("result_vars.pkl.gz", "wb") as f:
    pickle.dump(plot_blob, f, protocol=pickle.HIGHEST_PROTOCOL)




import pandas as pd

def _val(v, default=0.0):
    try: return float(v.X)
    except Exception:
        try: return float(v)
        except Exception: return float(default)

def _get_h_for(tr):
    if 'h' in globals():
        H = globals()['h']
        try:   return _val(H[tr], 1.0)
        except: return _val(H, 1.0)  # 스칼라일 때
    return 1.0

def _get_z_for(tr, o, d):
    # R_ok 열차는 z 변수를 만들지 않으므로 "전부 제공"으로 본다
    if tr in R_ok:
        return 1.0
    if 'z' in globals():
        Z = globals()['z']
        try:   return _val(Z[tr, o, d], 0.0)
        except: return 0.0
    return 0.0

def _get_delta_for(tr, o, d):
    if 'delta' in globals():
        D = globals()['delta']
        try:   return _val(D[tr, o, d], 0.0)
        except: return 0.0
    return 0.0

def _reroute_cost_for(tr):
    if 'u_eid' in globals():
        U = globals()['u_eid']
        try:   return sum(_val(v, 0.0) for (tr2, e), v in U.items() if tr2 == tr)
        except: return 0.0
    return 0.0

# --- (NEW) q_tot 접근 통일: 있으면 사용, 없으면 즉석 계산
if 'q_tot_map' in globals() and isinstance(q_tot_map, dict):
    _Q = q_tot_map
else:
    _Q = {tr: sum(q for *_, q in demand.get(tr, [])) for tr in trains}

def loss_breakdown_for_train(tr):
    _w1 = globals().get('w1', 0.0)
    _w2 = globals().get('w2', 0.0)
    _w3 = globals().get('w3', 0.0)
    _W_ROUTE = globals().get('W_ROUTE', 0.0)

    # 1) 취소 비용
    h_tr = _get_h_for(tr)
    C_cancel = _w1 * _Q.get(tr, 0.0) * (1.0 - h_tr)

    # 2) 미제공 + 3) 지연 비용
    C_unserved = 0.0
    C_delay    = 0.0
    for (o, d, q) in demand.get(tr, []):
        zval = _get_z_for(tr, o, d)
        dval = _get_delta_for(tr, o, d)
        C_unserved += _w2 * q * (1.0 - zval)
        C_delay    += _w3 * q * dval

    # 4) 재루팅 벌점
    C_reroute = _W_ROUTE * _reroute_cost_for(tr)

    return {
        "Train": tr,
        "C_total": C_cancel + C_unserved + C_delay + C_reroute,
        "C_cancel": C_cancel,
        "C_unserved": C_unserved,
        "C_delay": C_delay,
        "C_reroute": C_reroute,
    }

rows = [loss_breakdown_for_train(tr) for tr in trains]
df_loss = pd.DataFrame(rows).sort_values("C_total", ascending=False)
display(df_loss)
